In [ ]:
#!pip install torch torchvision torchaudio
#!pip install captum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from captum.attr import IntegratedGradients, LayerConductance, NeuronConductance

%matplotlib inline

In [ ]:
# load dataset
ds = datasets.fetch_california_housing()
X = ds.data.astype(np.float32)
y = ds.target.astype(np.float32)

# remove very cheap or very expensive homes (saturates =< 0.15 or >= 5)
ind = (y > 0.15) & (y < 5)
X = X[ind,:]
y = y[ind]

# transform target - more Gaussian
y = np.log(y)

# scale input attributes
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# split data into train and test - !!! added valid dataset for pytorch
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.33, random_state=0)

In [ ]:
# info about the dataset
print(ds.DESCR)

In [ ]:
# tensorize data
ts_X_train, ts_y_train, ts_X_valid, ts_y_valid, ts_X_test, ts_y_test = map(
    torch.tensor, (X_train, y_train.reshape(-1,1), X_valid, y_valid.reshape(-1,1), X_test, y_test.reshape(-1,1))
)

In [ ]:
model = torch.load('nonlin_reg.nn')
model

# Feature Attribution

In [ ]:
# https://captum.ai/tutorials/House_Prices_Regression_Interpret

In [ ]:
# use just a subset of the test dataset
n_smp = 500
ind = np.random.randint(0,ts_X_test.shape[0]-1,n_smp)   

In [ ]:
# Feature Attribution
ig = IntegratedGradients(model)      
ig_attr_test = ig.attribute(ts_X_test[ind,:], n_steps=50)

In [ ]:
# Plotting Feature Attribution

x_axis_data = np.arange(ts_X_test.shape[1])
x_axis_data_labels = list(map(lambda idx: ds.feature_names[idx], x_axis_data))

ig_attr_test_avg = np.abs(ig_attr_test.detach().numpy()).mean(0)

# Plot
plt.figure(figsize=(20, 10))

ax = plt.subplot()
ax.set_title('Feature Attributions')
ax.set_ylabel('Attributions')

FONT_SIZE = 15
plt.rc('font', size=FONT_SIZE)            # fontsize of the text sizes
plt.rc('axes', titlesize=FONT_SIZE)       # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)       # fontsize of the x and y labels
plt.rc('legend', fontsize=FONT_SIZE - 4)  # fontsize of the legend

ax.bar(x_axis_data, ig_attr_test_avg, align='center',  color='blue')
ax.autoscale_view()
plt.tight_layout()

ax.set_xticks(x_axis_data)
ax.set_xticklabels(x_axis_data_labels)

plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(X_test[:,7], X_test[:,6], s=20, c=y_test, cmap='seismic_r')
plt.title('Median House Value', fontsize=15)
plt.xlabel('feat1', fontsize=15)
plt.ylabel('feat2', fontsize=15)
plt.colorbar()

![California](https://geology.com/cities-map/map-of-california-cities.gif)

# Layer Attribution

In [ ]:
model

In [ ]:
# Compute the attributions of the output with respect to the inputs of specified layer

# Task 1: run with layer_idx = 9
# Task 2: run with layer_idx = 6
layer_idx = 9

lc = LayerConductance(model, model[layer_idx])
lc_attr_test = lc.attribute(ts_X_test[ind,:], n_steps=100, attribute_to_layer_input=True)

# get layer weights 
layer_weight = model[layer_idx].weight

In [ ]:
# Plotting Layer Attribution

plt.figure(figsize=(20, 10))
x_axis_data = np.arange(lc_attr_test.shape[1])

y_axis_lc_attr_test = np.abs(lc_attr_test.detach().numpy()).mean(0)
y_axis_layer_weight = np.abs(layer_weight[0].detach().numpy())

width = 0.25
legends = ['Attributions','Weights']
x_axis_labels = [ f'{i}' for i in range(len(y_axis_layer_weight))]

ax = plt.subplot()
ax.set_title('Aggregated neuron importances and learned weights in the specified linear layer of the model')
ax.set_ylabel('Attributions')
ax.set_xlabel('Neuron')

ax.bar(x_axis_data + width, y_axis_lc_attr_test, width, align='center', alpha=0.5, color='red')
ax.bar(x_axis_data + 2 * width, y_axis_layer_weight, width, align='center', alpha=0.5, color='green')
plt.legend(legends, loc=2, prop={'size': 20})
ax.autoscale_view()
plt.tight_layout()

ax.set_xticks(x_axis_data + 0.4)
ax.set_xticklabels(x_axis_labels)

plt.show()

# Neuron Attribution

In [ ]:
# https://captum.ai/tutorials/Titanic_Basic_Interpret

In [ ]:
model

In [ ]:
# Compute the attributions of the inputs with respect to a specified neuron output
layer_idx = 6

# Task 1: run with neuron_idx = 0
# Task 2: run with neuron_idx = 1
neuron_idx = 0

neuron_cond = NeuronConductance(model, model[layer_idx])
neuron_attr_test = neuron_cond.attribute(ts_X_test[ind,:], neuron_selector=neuron_idx)

In [ ]:
# Plotting Feature Attribution

x_axis_data = np.arange(ts_X_test.shape[1])
x_axis_data_labels = list(map(lambda idx: ds.feature_names[idx], x_axis_data))

neuron_attr_test_avg = np.abs(neuron_attr_test.detach().numpy()).mean(0)

# Plot
plt.figure(figsize=(20, 10))

ax = plt.subplot()
ax.set_title('Neuron Attributions')
ax.set_ylabel('Attributions')

FONT_SIZE = 15
plt.rc('font', size=FONT_SIZE)            # fontsize of the text sizes
plt.rc('axes', titlesize=FONT_SIZE)       # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)       # fontsize of the x and y labels
plt.rc('legend', fontsize=FONT_SIZE - 4)  # fontsize of the legend

ax.bar(x_axis_data, neuron_attr_test_avg, align='center',  color='blue')
ax.autoscale_view()
plt.tight_layout()

ax.set_xticks(x_axis_data)
ax.set_xticklabels(x_axis_data_labels)

plt.show()